<a href="https://colab.research.google.com/github/muthuraman2002/langGraph_learning/blob/main/langGraph_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
! pip install langgraph chromadb transformers sentence-transformers langchain langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [10]:
try:
  from langgraph.graph import MessageGraph
  from langchain.vectorstores import Chroma
  from langchain.embeddings import HuggingFaceEmbeddings
  from langchain.llms import HuggingFacePipeline
  from transformers import pipeline
except Exception as e:
  print(f"Please install the required packages.{e}")

In [15]:
try:

# Initialize Hugging Face Embeddings
  embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Initialize ChromaDB (in-memory for simplicity)
  vectorstore = Chroma(persist_directory="./chromadb", embedding_function=embeddings)
  # print(vectorstore,'chorma')
# Add some sample documents to ChromaDB
  documents = [
    "LangGraph makes it easy to design AI workflows.",
    "ChromaDB is a vector database optimized for fast retrieval.",
    "LangChain can integrate with ChromaDB for advanced use cases."
  ]
  vectorstore.add_texts(documents)
  # print(vectorstore)
# Define a local LLM pipeline (replace 'gpt2' with a more advanced model if available)
  local_pipeline = pipeline("text-generation", model="gpt2", device=-1)  # Use -1 for CPU or specify a GPU ID
  llm = HuggingFacePipeline(pipeline=local_pipeline)

# Create a LangGraph Workflow
  graph = MessageGraph()

# Add a retrieval node to fetch relevant documents from ChromaDB
  retriever_node = graph.add_node(
      name="retriever",
      func=lambda query: vectorstore.similarity_search(query, k=3),
      inputs=["query"],
      outputs=["retrieved_docs"]
  )
  # print(retriever_node)
# Add an LLM node to process the retrieved documents and generate a response
  llm_node = graph.add_node(
      name="llm",
      func=lambda retrieved_docs: llm(f"Answer the question based on: {retrieved_docs}"),
      inputs=["retrieved_docs"],
      outputs=["response"]
  )

# Link the nodes
  graph.link(retriever_node, llm_node)

# Execute the workflow
  query = "What is LangGraph used for?"
  result = graph.run(inputs={"query": query})

# Display the result
  print("Q:", query)
  print("A:", result["response"])
except Exception as e:
  print(f"An error occurred: {e}")

Device set to use cpu


An error occurred: StateGraph.add_node() got an unexpected keyword argument 'name'
